![alt text](https://portalsyngenta.com.br/sites/default/files/syngenta_pragas0089_842x384.jpg)

## Etapa 1: Instalação das dependências

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("You are using TensorFlow version", tf.__version__)
if len(tf.config.list_physical_devices('GPU')) > 0:
  print("You have a GPU enabled.")
else:
  print("Enable a GPU before running this notebook.")

You are using TensorFlow version 2.2.0-rc2
You have a GPU enabled.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!! nvidia-smi

*Biblioteca para melhorar as barras de progresso

In [4]:
!pip install tqdm

## Etapa 2: Pré-processamento

### Importação das bibliotecas

In [0]:
import os
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline


### Configurando os caminhos (paths)

In [0]:
dataset_path_new = "/content/drive/My Drive/datasets/soja"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construindo o modelo

### Carregando o modelo pré-treinado (MobileNetV2)

In [0]:
img_shape = (128, 128, 3)

In [9]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, 
                                               include_top = False,
                                               weights = "imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [0]:
base_model.summary()

In [0]:
tf.keras.utils.plot_model(base_model, show_shapes=True, dpi=70)

### Congelando o modelo base

In [0]:
base_model.trainable = False

### Definindo o cabeçalho personalizado da rede neural

In [13]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

Reduzindo o número de parâmetros (4*4*1280 > 20k)

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [15]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units = 10, activation = "softmax")(global_average_layer)

### Definindo o modelo

In [0]:
model = tf.keras.models.Model(inputs = base_model.input, outputs = prediction_layer)

In [0]:
model.summary()

### Compilando o modelo

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss="categorical_crossentropy", 
              metrics = ["accuracy"])

### Criando geradores de dados (Data Generators)

Redimensionando as imagens

Grandes arquiteturas treinadas suportam somente alguns tamanhos pré-definidos.

Por exemplo: MobileNet suporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [0]:
image_size = 128
batch_size = 32

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [21]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(image_size,image_size), batch_size=batch_size, class_mode="categorical")
validation_generator = data_gen_train.flow_from_directory(validation_dir, target_size=(image_size,image_size), batch_size=batch_size, class_mode="categorical")

Found 259 images belonging to 10 classes.
Found 58 images belonging to 10 classes.


### Treinando o modelo

In [0]:
import numpy

In [0]:
epochs = 25
steps_per_epoch = numpy.ceil(train_generator.n / batch_size)
validation_steps = numpy.ceil(validation_generator.n / batch_size)

In [0]:
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=validation_steps)

### Avaliação do modelo de transferência de aprendizagem

In [27]:
valid_loss, valid_accuracy = model.evaluate_generator(validation_generator,steps=60)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [28]:
valid_accuracy

0.7591953873634338

### Salvar modelo e converter em TFLite

In [29]:
saved_model_dir = '/content/TFLite'
tf.saved_model.save(model, saved_model_dir)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /content/TFLite/assets


In [0]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()


In [0]:
with open('soja.tflite', 'wb') as f:
  f.write(tflite_model)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels)